In [152]:
import cv2
import numpy as np

cap = cv2.VideoCapture("../../volleyball_video_full_2.mp4")
# cap = cv2.VideoCapture("volleyball_5_mins.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

scale_factor = 0.5
new_width = int(width * scale_factor)
new_height = int(height * scale_factor)

out = cv2.VideoWriter("volleyball_tracking.avi", cv2.VideoWriter_fourcc(*'XVID'), fps, (new_width, new_height))

# median backSub
# background_frames = 50
# frame_samples = []
# for _ in range(background_frames):
#     ret, frame = cap.read()
#     if not ret:
#         break
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame_samples.append(frame)

# background = np.median(np.array(frame_samples), axis=0).astype(np.uint8)

subtractor = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=500, detectShadows=True)
# subtractor = cv2.createBackgroundSubtractorMOG2(history=500, detectShadows=True, varThreshold=100)
# subtractor.setNMixtures(5)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    frame = cv2.resize(frame, (new_width, new_height))  # for performance

    # movement_mask = np.any(np.abs(frame_rgb - background) > 20, axis=2).astype(np.uint8) * 255  # Convert to 255 for display
    
    movement_mask = subtractor.apply(frame)
    movement_mask[movement_mask == 127] = 0  # Eliminate 127 values to 0
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)))
    movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_RECT, (8, 12)))
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 15)))
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_DILATE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    
    # detect and remove long lines
    # edges = cv2.Canny(movement_mask, 50, 150, apertureSize=3)
    # line_mask = np.zeros_like(movement_mask)
    # lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=500, minLineLength=150, maxLineGap=100)
    # if lines is not None:
    #     for line in lines:
    #         x1, y1, x2, y2 = line[0]
    #         line_length = np.hypot(x2 - x1, y2 - y1)
    #         if line_length > 1000:  # adjust threshold as needed
    #             cv2.line(movement_mask, (x1, y1), (x2, y2), 0, thickness=20, lineType=cv2.LINE_8)

    
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
    
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2eq.MORPH_ELLIPSE, (3, 3)))
    # movement_mask = cv2.morphologyEx(movement_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))

    # color_mask_value = 0.5 * frame_rgb[:, :, 0] + 0.5 * frame_rgb[:, :, 1] - frame_rgb[:, :, 2]
    # color_mask = ((color_mask_value - np.min(color_mask_value)) / (np.max(color_mask_value) - np.min(color_mask_value)) > 0.77).astype(np.uint8) * 255
    
    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([10, 0, 0])
    upper_yellow = np.array([50, 255, 255])
    color_mask = cv2.inRange(frame_hsv, lower_yellow, upper_yellow)
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))
    color_mask = cv2.morphologyEx(color_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))
    
    contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ball_mask = np.zeros_like(frame[:, :, 0])  # create a new binary mask for the ball
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        # cv2.circle(frame, center, 10, (0, 0, 255), -1)
        # cv2.circle(frame, center, int(radius*0.9), (0, 0, 255), -1)
        cv2.circle(ball_mask, center, int(radius)+1, 255, -1)
        
        
    contours_movement, _ = cv2.findContours(movement_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_movement = sorted(contours_movement, key=cv2.contourArea, reverse=True)[:6]  # Keep only the 5 largest contours
    movement_mask = np.zeros_like(movement_mask)  # Create a new mask for the largest contours
    cv2.drawContours(movement_mask, contours_movement, -1, 255, thickness=cv2.FILLED)  # Fill the largest contours in the new mask
    # cv2.drawContours(frame, contours_movement, -1, (0, 255, 0), 2)  # draw contours on the original frame in green

    players_only = cv2.bitwise_and(frame, frame, mask=movement_mask)
    ball_only = cv2.bitwise_and(frame, frame, mask=ball_mask)
    ball_and_players = cv2.bitwise_or(players_only, ball_only)

    frame_resized = cv2.resize(frame, (new_width, new_height))
    players_only_resized = cv2.resize(players_only, (new_width, new_height))
    ball_only_resized = cv2.resize(ball_only, (new_width, new_height))
    ball_and_players_resized = cv2.resize(ball_and_players, (new_width, new_height))
    movement_mask_resized = cv2.resize(movement_mask, (new_width, new_height))
    color_mask_resized = cv2.resize(color_mask, (new_width, new_height))
    ball_mask_resized = cv2.resize(ball_mask, (new_width, new_height))

    # cv2.imshow("Movement Mask", movement_mask_resized)
    # cv2.imshow("Color Mask", color_mask_resized)
    # cv2.imshow("Final Mask (Movement & Color)", mask_resized)
    cv2.imshow("Original", frame_resized)
    # cv2.imshow("Ball Mask", ball_mask_resized)
    
    # DEMO SHOW
    cv2.imshow("Ball Only", ball_only_resized)
    cv2.imshow("Players Only", players_only_resized)
    cv2.imshow("Ball + Players", ball_and_players_resized)
    
    out.write(frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()


In [122]:
import cv2
import numpy as np

# Load 250th frame from the video
cap = cv2.VideoCapture('volleyball_5_mins.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES, 250)
ret, frame = cap.read()
cap.release()

if not ret:
    print("Failed to read frame.")
    exit()

# Resize for easier display (optional)
frame = cv2.resize(frame, (960, 540))

# Convert to HSV
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# Create a window
cv2.namedWindow('Trackbars')

# Callback for trackbar (does nothing but required)
def nothing(x):
    pass

# Create trackbars for HSV thresholds
cv2.createTrackbar('Lower H', 'Trackbars', 0, 180, nothing)
cv2.createTrackbar('Lower S', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('Lower V', 'Trackbars', 200, 255, nothing)
cv2.createTrackbar('Upper H', 'Trackbars', 180, 180, nothing)
cv2.createTrackbar('Upper S', 'Trackbars', 40, 255, nothing)
cv2.createTrackbar('Upper V', 'Trackbars', 255, 255, nothing)

while True:
    # Get current positions of all trackbars
    lh = cv2.getTrackbarPos('Lower H', 'Trackbars')
    ls = cv2.getTrackbarPos('Lower S', 'Trackbars')
    lv = cv2.getTrackbarPos('Lower V', 'Trackbars')
    uh = cv2.getTrackbarPos('Upper H', 'Trackbars')
    us = cv2.getTrackbarPos('Upper S', 'Trackbars')
    uv = cv2.getTrackbarPos('Upper V', 'Trackbars')

    lower = np.array([lh, ls, lv])
    upper = np.array([uh, us, uv])

    # Create mask based on current HSV bounds
    mask = cv2.inRange(hsv_frame, lower, upper)
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Show original frame, mask, and result side-by-side
    cv2.imshow('Original', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Filtered', result)

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):  # Press ESC or q to exit
        break

cv2.destroyAllWindows()


In [58]:
import cv2
import numpy as np

# cap = cv2.VideoCapture("volleyball_video_full_2.mp4")
cap = cv2.VideoCapture("volleyball_5_mins.mp4")

knn = cv2.createBackgroundSubtractorKNN(history=0, dist2Threshold=200, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 360))

    fg_mask_knn = knn.apply(frame_resized)

    fg_mask = np.where(fg_mask_knn == 255, 255, 0).astype(np.uint8)

    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel, iterations=2)

    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.4 * dist_transform.max(), 255, 0)
    sure_fg = sure_fg.astype(np.uint8)

    unknown = cv2.subtract(sure_bg, sure_fg)

    _, markers = cv2.connectedComponents(sure_fg)

    markers = markers + 1

    markers[unknown == 255] = 0

    markers = cv2.watershed(frame_resized, markers)

    for marker_id in np.unique(markers):
        if marker_id <= 1:
            continue
        mask = np.uint8(markers == marker_id)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue
        c = max(contours, key=cv2.contourArea)
        if cv2.contourArea(c) < 300:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('Foreground Mask (KNN)', fg_mask)
    cv2.imshow('Segmented Players', frame_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [56]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture("volleyball_video_full_2.mp4")

# Background subtractor
knn = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=500, detectShadows=True)
# knn = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=100, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_resized = cv2.resize(frame, (640, 360))

    # Apply KNN background subtraction
    fg_mask_knn = knn.apply(frame_resized)

    # Remove shadows (127) and clean mask
    fg_mask = np.where(fg_mask_knn == 255, 255, 0).astype(np.uint8)

    # Morphological closing to merge close regions (players close together)
    fg_closed = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (1, 2)))

    # Optional: remove small noise
    # fg_closed = cv2.morphologyEx(fg_closed, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Find contours
    contours, _ = cv2.findContours(fg_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank mask to draw filled player shapes
    player_mask = np.zeros_like(fg_closed)

    for contour in contours:
        if cv2.contourArea(contour) < 100:
            continue  # filter out small noise
        cv2.drawContours(player_mask, [contour], -1, 255, thickness=cv2.FILLED)

    # Optional: visualize filled shapes on original frame
    result = frame_resized.copy()
    result[player_mask == 255] = [0, 255, 0]  # overlay green for player blobs

    # Show outputs
    cv2.imshow('Foreground Mask (cleaned)', fg_closed)
    cv2.imshow('Foreground Mask (KNN)', fg_mask_knn)
    cv2.imshow('Foreground Mask', fg_mask)
    cv2.imshow('Player Shapes', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
